# ETL Manager Migration

In this tutorial we are going to:
1. set up a database using etl_manager schemas
2. query that database
3. Use our Converters to convert the old etl_manager schemas into our new ones
4. redeploy the database using our new schemas
5. query the database again to make sure all is gravy

## Init / Basic setup

In [16]:
import os
import shutil
import json

import pandas as pd
import awswrangler as wr

import boto3
from mojap_metadata import Metadata
from mojap_metadata.converters.glue_converter import GlueConverter
from mojap_metadata.converters.etl_manager_converter import EtlManagerConverter
from etl_manager.meta import read_database_folder

In [2]:
# setup your own testing area (set foldername = GH username)
foldername = "isichei" # GH username

In [3]:
region = "eu-west-1"
bucketname = "alpha-everyone"
db_name = f"aws_example_{foldername}"
db_base_path = f"s3://{bucketname}/{foldername}/database"
s3_base_path = f"s3://{bucketname}/{foldername}/"
if wr.s3.list_objects(s3_base_path):
    print("deleting objs")
    wr.s3.delete_objects(s3_base_path)

deleting objs


## 1. Create DB using etl_manager

Define the database config and then the schemas for the 3 tables (`department`, `sales` and `employees`) these 3 tables can be found in the `data/` folder. 

Write them into a folder named `meta_data/` which is standard with projects that use `etl_manager`.

In [4]:
database = {
    "description": "Test database for etl_manager migration example",
    "name": db_name,
    "bucket": bucketname,
    "base_folder": db_base_path.replace(f"s3://{bucketname}/", "")
}

department = {
    "$schema": "https://raw.githubusercontent.com/moj-analytical-services/etl_manager/master/etl_manager/specs/table_schema.json",
    "name": "department",
    "description": "A lookup table for each department",
    "data_format": "csv",
    "columns": [
        {
            "name": "department_id",
            "type": "int",
            "description": "ID of the department"
        },
        {
            "name": "department_name",
            "type": "character",
            "description": "Name of the department"
        }
    ],
    "location": "department"
}

sales = {
    "$schema": "https://raw.githubusercontent.com/moj-analytical-services/etl_manager/master/etl_manager/specs/table_schema.json",
    "name": "sales",
    "description": "quarterly sales of each employee",
    "data_format": "csv",
    "columns": [
        {
            "name": "employee_id",
            "type": "int",
            "description": "The employee number"
        },
        {
            "name": "qtr",
            "type": "int",
            "description": "The quarter when the sales were made"
        },
        {
            "name": "sales",
            "type": "double",
            "description": "The total number of sales the employee made that quarter"
        }
    ],
    "location": "sales"
}

employees = {
    "$schema": "https://raw.githubusercontent.com/moj-analytical-services/etl_manager/master/etl_manager/specs/table_schema.json",
    "name": "employees",
    "description": "A table of employees",
    "data_format": "csv",
    "columns": [
        {
            "name": "employee_id",
            "type": "int",
            "description": "The employee number"
        },
        {
            "name": "sex",
            "type": "character",
            "description": "Sex of the employee"
        },
        {
            "name": "forename",
            "type": "character",
            "description": "First name of the employee"
        },
        {
            "name": "surname",
            "type": "character",
            "description": "Last name of the employee"
        },
        {
            "name": "department_id",
            "type": "int",
            "description": "Department ID which the employee belongs to"
        },
        {
            "name": "manager_id",
            "type": "int",
            "description": "The Employee number of the person who manages this employee"
        }
    ],
    "location": "employees"
}

Write each of these to a folder to mimic a standard etl_manager project

In [5]:
meta_dir = 'meta_data'
if os.path.exists(meta_dir):
    shutil.rmtree(meta_dir)
os.makedirs(meta_dir)

for m in [department, sales, employees]:
    with open(os.path.join(meta_dir, f"{m['name']}.json"), "w") as f:
        json.dump(m, f)
        
with open("meta_data/database.json", "w") as f:
    json.dump(database, f)

Now we have our etl_manager metadata schemas in a folder called `meta_data`

In [6]:
os.listdir(meta_dir)

['sales.json', 'database.json', 'department.json', 'employees.json']

Upload the data to S3 (using wrangler) and then create a database (using etl_manager)

In [7]:
db = read_database_folder("meta_data/")

for table_name in db.table_names:
    df = pd.read_csv(f"data/{table_name}.csv")
    tab = db.table(table_name)
    tab_s3_path = os.path.join(
        db.s3_database_path,
        tab.location,
        tab.name + ".csv"
    )
    wr.s3.to_csv(df, tab_s3_path, index=False)

In [8]:
db.create_glue_database(delete_if_exists=True)

## 2. Query the database

Using wrangler

In [9]:
# employees table
wr.athena.read_sql_query(
    f"SELECT * FROM {db_name}.employees",
    database=db_name,
    ctas_approach=False
).head()

,employee_id,sex,forename,surname,department_id,manager_id
0,1,M,Dexter,Mitchell,1,17
1,2,F,Summer,Bennett,1,17
2,3,M,Pip,Carter,1,17
3,4,F,Bella,Long,1,17
4,5,F,Lexie,Perry,<NA>,17


In [10]:
# sales table
wr.athena.read_sql_query(
    f"SELECT * FROM {db_name}.sales",
    database=db_name,
    ctas_approach=False
).head()

,employee_id,qtr,sales
0,1,1,768.17
1,2,1,391.98
2,3,1,406.36
3,4,1,816.25
4,5,1,437.05


In [28]:
# department table
wr.athena.read_sql_query(
    f"SELECT * FROM {db_name}.department",
    database=db_name,
    ctas_approach=False
).head()

,department_id,department_name
0,1,Sales
1,2,Admin
2,3,Management
3,4,Technical
4,5,Maintenance


## 3. Convert etl_manager schemas into our new schemas

Use our `EtlManagerConverter` to convert the old schemas into new `Metadata` schemas and write the new schemas to a new folder.

In [12]:
new_meta_dir = 'metadata_schemas'
if os.path.exists(new_meta_dir):
    shutil.rmtree(new_meta_dir)
os.makedirs(new_meta_dir)

emc = EtlManagerConverter()

for table_name in db.table_names:
    table_meta = emc.generate_to_meta(db.table(table_name))
    table_meta.to_json(f"{new_meta_dir}/{table_name}.json")

In [13]:
# lets take a look at one of the new schemas
table_meta.to_dict()

{'name': 'department',
 'description': 'A lookup table for each department',
 'location': 'department',
 'file_format': 'csv',
 'columns': [{'name': 'department_id',
   'type': 'int32',
   'description': 'ID of the department'},
  {'name': 'department_name',
   'type': 'string',
   'description': 'Name of the department'}],
 '_converted_from': 'etl_manager',
 '$schema': '',
 'sensitive': False,
 'primary_key': [],
 'partitions': []}

## 4. redeploy the database using our new schemas

First we need to delete the table schemas (using `aws-wrangler`) then recreate the schemas using our new metadata and the (`GlueConverter` and `boto3` glue client).

In [30]:
# Delete tables
for t in ["employees", "department", "sales"]:
    dropped = wr.catalog.delete_table_if_exists(database = db_name, table=t)
    if dropped:
        print(f"Table {t} deleted")

Table employees deleted
Table department deleted
Table sales deleted


In [31]:
glue_client = boto3.client("glue")

new_meta_dir = 'metadata_schemas'

gc = GlueConverter()

# set db name and base path so we don't have to set it everytime
gc.options.default_db_name = db_name 
gc.options.default_db_base_path = db_base_path

for fn in os.listdir(new_meta_dir):
    meta = Metadata.from_json(f"{new_meta_dir}/{fn}")
    boto_dict = gc.generate_from_meta(meta)
    glue_client.create_table(**boto_dict)

sales
department
employees


## 5. query the database again to make sure all is gravy

In [33]:
# employees table
wr.athena.read_sql_query(
    f"SELECT * FROM {db_name}.employees",
    database=db_name,
    ctas_approach=False
).head()

,employee_id,sex,forename,surname,department_id,manager_id
0,1,M,Dexter,Mitchell,1,17
1,2,F,Summer,Bennett,1,17
2,3,M,Pip,Carter,1,17
3,4,F,Bella,Long,1,17
4,5,F,Lexie,Perry,<NA>,17


In [34]:
# sales table
wr.athena.read_sql_query(
    f"SELECT * FROM {db_name}.sales",
    database=db_name,
    ctas_approach=False
).head()

,employee_id,qtr,sales
0,1,1,768.17
1,2,1,391.98
2,3,1,406.36
3,4,1,816.25
4,5,1,437.05


In [35]:
# department table
wr.athena.read_sql_query(
    f"SELECT * FROM {db_name}.department",
    database=db_name,
    ctas_approach=False
).head()

,department_id,department_name
0,1,Sales
1,2,Admin
2,3,Management
3,4,Technical
4,5,Maintenance
